In [1]:
import tqdm
from langchain import PromptTemplate
from utils import load_pickle_file, load_cta_dataset_column, save_pickle_file, load_cta_dataset, calculate_f1_scores, decimal, map_answers_column, map_cta_labels, map_sportstables
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4" #4,5,6,7

In [ ]:
datasets = ["sotabv2", "t2dv2-webtables", "sportstables"]

# StableBeluga7B
model_name = "stabilityai/StableBeluga-7B"
mod = "stablebeluga7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="hf_cache/")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", cache_dir="hf_cache/")

# SOLAR
# model_name = "upstage/SOLAR-0-70b-16bit"
# mod = "solar"
# tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="hf_cache/")
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_8bit=True, device_map="auto", cache_dir="hf_cache/", temperature=0, do_sample=True)

## Table-prompts experiments

In [5]:
zero_template = """Answer the question based on the task and instructions below.
Task: Classify the columns of a given table with only one of the following classes that are separated with comma: {labels_joined}.
Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, select a class that best represents the meaning of all cells in the column. 4. Answer with the selected class for each columns with the format Column1: class. 5. Answer only with labels from the provided label set!
Table:
{input_string}
Class:"""

In [10]:
few_template = """Answer the question based on the task and instructions below.
Task: Classify the columns of a given table with only one of the following classes that are separated with comma: {labels_joined}.
Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, select a class that best represents the meaning of all cells in the column. 4. Answer with the selected class for each columns with the format Column1: class. 5. Answer only with labels from the provided label set!
{examples}
Table:
{input_string}Class:"""

In [ ]:
for dataset in datasets:
    # Load dataset
    examples, labels, test_table_type_labels, train_examples, train_example_labels, train_table_type_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset(dataset,"")
    all_labels = [labels_to_text[l] for l in labels_to_text]

    examples_demonstrations = load_pickle_file(f"embeddings/examples_demonstrations_{dataset}.pkl")
    cc_demonstrations = load_pickle_file(f"embeddings/cc_examples_demonstrations_{dataset}.pkl")

    # Zero-shot
    prompt = PromptTemplate(template=zero_template, input_variables=['input_string', 'labels_joined'])
            
    prompts = []
    model_answers = []
            
    for example in tqdm.tqdm(examples, total=len(examples)):
        text_prompt = prompt.format(input_string=example.strip(), labels_joined=labels_joined)
        prompts.append(text_prompt)

        inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
        model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))

    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-0-shot.pkl", model_answers)
    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-0-shot-prompts.pkl", prompts)

    # Few-shot
    for j in [1, 3]:
        prompts = []
        model_answers = []
        prompt = PromptTemplate(template=few_template, input_variables=['input_string', 'examples', 'labels_joined'])

        for example in tqdm.tqdm(examples, total=len(examples)):

            random_examples = """"""
            for i in range(0,j):
                index = random.randint(0, len(train_examples)-1)
                random_examples += f"""Table:\n{train_examples[index]}Class:\n{train_example_labels[index]}\n"""
            random_examples = random_examples.strip()

            text_prompt = prompt.format(input_string=example, examples=random_examples, labels_joined=labels_joined)
            prompts.append(text_prompt)

            inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
            output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
            model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))
            
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-{j}-shot.pkl", model_answers)
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-{j}-shot-prompts.pkl", prompts)

    # Few-shot: similar
    prompts = []
    model_answers = []
    prompt = PromptTemplate(template=few_template, input_variables=['input_string', 'examples', 'labels_joined'])

    for i, example in tqdm.tqdm(enumerate(examples), total=len(examples)):

        random_examples = """"""
        for index in examples_demonstrations[i][-3:]:
            random_examples += f"""Table:\n{train_examples[index]}Class:\n{train_example_labels[index]}\n"""    
        random_examples = random_examples.strip()

        text_prompt = prompt.format(input_string=example, examples=random_examples, labels_joined=labels_joined)
        prompts.append(text_prompt)

        inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
        model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))

    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-3-similar-shot.pkl", model_answers)
    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-3-similar-shot-prompts.pkl", prompts)
    
    # Few-shot: corner-cases
    prompts = []
    model_answers = []
    prompt = PromptTemplate(template=few_template, input_variables=['input_string', 'examples', 'labels_joined'])

    for i, example in tqdm.tqdm(enumerate(examples), total=len(examples)):

        random_examples = """"""
        for index in cc_demonstrations[i]:
            random_examples += f"""Table:\n{train_examples[index]}Class:\n{train_example_labels[index]}\n"""    
        random_examples = random_examples.strip()

        text_prompt = prompt.format(input_string=example, examples=random_examples, labels_joined=labels_joined)
        prompts.append(text_prompt)

        inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
        model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))

    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-4-cc-shot.pkl", model_answers)
    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-table-4-cc-shot-prompts.pkl", prompts)


## Column-prompts experiments

In [ ]:
zero_template = """Answer the question based on the task and instructions below.
Task: Classify the column given to you into only one of these classes that are separated with comma: {labels_joined}.
Instructions: 1. Look at the column and the classes given to you. 2. Examine the values of the column. 3. Select a class that best represents the meaning of the column. 4. Answer with the selected class.
Column: {input_string}
Class:"""

In [ ]:
few_template = """Answer the question based on the task and instructions below.
Task: Classify the column given to you into only one of these classes that are separated with comma: {labels_joined}.
Instructions: 1. Look at the column and the classes given to you. 2. Examine the values of the column. 3. Select a class that best represents the meaning of the column. 4. Answer with the selected class.
{examples}
Column: {input_string}
Class:"""

In [ ]:
for dataset in datasets:
    # Load dataset
    examples, labels, train_examples, train_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset_column(dataset,"")
    all_labels = [labels_to_text[l] for l in labels_to_text]

    examples_demonstrations = load_pickle_file(f"embeddings/examples_demonstrations_{dataset}-column.pkl")

    # Zero-shot
    prompt = PromptTemplate(template=zero_template, input_variables=['input_string', 'labels_joined'])
        
    prompts = []
    model_answers = [] 

    for example in tqdm.tqdm(examples, total=len(examples)):
        text_prompt = prompt.format(input_string=example.strip(), labels_joined=labels_joined)
        prompts.append(text_prompt) 
        
        inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
        model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))        

    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-column-0-shot.pkl", model_answers)
    save_pickle_file(f"predictions/{dataset}/{mod}/prompt-column-0-shot-prompts.pkl", prompts)

    for j in [1,3]:
        prompts = []
        model_answers = []

        prompt = PromptTemplate(template=few_template, input_variables=['input_string', 'examples', 'labels_joined'])

        for example in tqdm.tqdm(examples, total=len(examples)):

            random_examples = """"""
            for i in range(0,j):
                index = random.randint(0, len(train_examples)-1)
                random_examples += f"""Column: {train_examples[index]}\nClass: {train_labels[index]}\n"""
            random_examples = random_examples.strip()

            text_prompt = prompt.format(input_string=example, examples=random_examples, labels_joined=labels_joined)
            prompts.append(text_prompt)

            inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
            output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
            model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))

        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-column-{j}-shot.pkl", model_answers)
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-column-{j}-shot-prompts.pkl", prompts)

## Two-step

In [38]:
table_template = """Your task is to classify a table into one of these domains: {domains_list}.
Your instructions are: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. Decide the domain that best represents the table. 4. Answer with one domain.
{examples}
Classify this table: {input_string}
Answer:"""

template = """Answer the question based on the task and instructions below.
Task: Classify the columns of a given table with only one of the following classes that are separated with comma: {labels_joined}.
Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, select a class that best represents the meaning of all cells in the column. 4. Answer with the selected class for each columns with the format Column1: class. 5. Answer only with labels from the provided label set!
{examples}
Table:
{input_string}
Class:"""

In [40]:
def get_clean_table_prediction(table_pred, domains):
    cleaned_table_pred="-"
    for dom in domains:
        if dom in table_pred:
            cleaned_table_pred = dom
            break
    return cleaned_table_pred

In [ ]:
for dataset in datasets:
    # Load dataset
    examples, labels, test_table_type_labels, train_examples, train_example_labels, train_table_type_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset(dataset,"")
    all_labels = [labels_to_text[l] for l in labels_to_text]

    domains = set(train_table_type_labels)
    labels_dict = {}
    for dom in domains:
        f = open(f"../data/sportstables-labels/{dataset}_{dom}_labels.txt", 'r')
        t = [line.split('\n')[0] for line in f.readlines()]
        labels_dict[dom] = t

    domains_list = ", ".join(domains)

    for j in [0, 1, 3]:
        # Step 1
        table_prompts = []
        table_model_answers = []
        # Step 2
        prompts = []
        model_answers = []

        for example in tqdm.tqdm(examples, total=len(examples)):
            prompt = PromptTemplate(template=table_template, input_variables=['input_string', 'domains_list', 'examples'])

            random_examples = """"""

            for i in range(0,j):
                index = random.randint(0, len(train_examples)-1)
                random_examples += f"""Classify this table:{train_examples[index]}Class: {train_table_type_labels[index]}\n"""

            random_examples = random_examples.strip()

            text_prompt = prompt.format(input_string=example, examples=random_examples, domains_list=domains_list)
            table_prompts.append(text_prompt)

            inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
            output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
            answer = tokenizer.decode(output[0], skip_special_tokens=True)
            table_model_answers.append(answer)

            clean_prediction = get_clean_table_prediction(answer.replace(text_prompt, "").strip(), domains)

            prompt = PromptTemplate(template=template, input_variables=['input_string','labels_joined', 'examples'])

            random_examples = """"""

            if clean_prediction != "-":
                labels_dom = ", ".join([labels_to_text[l] for l in labels_dict[clean_prediction]])

                for m in range(0,j):
                    index = random.choice([j for j, e in enumerate(train_table_type_labels) if e == clean_prediction])
                    random_examples += f"""Table:\n{train_examples[index]}Class:\n{train_example_labels[index]}\n"""
                random_examples = random_examples.strip()

            else:
                labels_dom = labels_joined

            text_prompt = prompt.format(input_string=example, examples=random_examples, labels_joined=labels_dom)
            prompts.append(text_prompt)

            inputs = tokenizer(text_prompt, return_tensors="pt").to("cuda")
            output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
            model_answers.append(tokenizer.decode(output[0], skip_special_tokens=True))

        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-two-step-{j}-shot-step1.pkl", table_model_answers)
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-two-step-{j}-shot-step1-prompts.pkl", table_prompts)
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-two-step-{j}-shot-step2.pkl", model_answers)
        save_pickle_file(f"predictions/{dataset}/{mod}/prompt-two-step-{j}-shot-step2-prompts.pkl", prompts)

## Evaluation

In [ ]:
# Column prompt evaluation
print(f"\tPrecision)\tRecall\tMacro-F1\tMicro-F1\tOOV")
for nr in [0, 1, 3]:
    preds = load_pickle_file(f'predictions/{dataset}/{model}/prompt-column-{nr}-shot.pkl')
    prompts = load_pickle_file(f'predictions/{dataset}/{model}/prompt-column-{nr}-shot-prompts.pkl')

    preds = [pred.replace(prompts[i], "") for i,pred in enumerate(preds)]
    
    predictions, num = map_answers_column(preds,prompts)
    types = list(set(labels))
    types = types + ["-"] if "-" in predictions else types
    evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)
    
    print(f"{decimal(evaluation['Precision'])}\t{decimal(evaluation['Recall'])}\t{decimal(evaluation['Macro-F1'])}\t{decimal(evaluation['Micro-F1'])}\t{num}")

In [ ]:
# Table prompt evaluation
print(f"\tPrecision)\tRecall\tMacro-F1\tMicro-F1\tOOV")
for nr in [ 0, 1, 3, "3-similar", "4-cc"]:
    preds = load_pickle_file(f'icde-predictions/{dataset}/{model}/prompt-table-{nr}-shot.pkl')
    prompts = load_pickle_file(f'icde-predictions/{dataset}/{model}/prompt-table-{nr}-shot-prompts.pkl')

    preds = [pred.replace(prompts[i], "") for i,pred in enumerate(preds)]

    predictions, num = map_cta_labels(preds,prompts)
    types = list(set(labels))
    types = types + ["-"] if "-" in predictions else types
    evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)
    
    print(f"{decimal(evaluation['Precision'])}\t{decimal(evaluation['Recall'])}\t{decimal(evaluation['Macro-F1'])}\t{decimal(evaluation['Micro-F1'])}\t{num}")

In [ ]:
# SportsTables table prompt evaluation
print(f"\tPrecision)\tRecall\tMacro-F1\tMicro-F1\tOOV")
for nr in [ 0, 1, 3, "3-similar", "4-cc"]:
    preds = load_pickle_file(f'icde-predictions/{dataset}/{model}/prompt-table-{nr}-shot.pkl')
    prompts = load_pickle_file(f'icde-predictions/{dataset}/{model}/prompt-table-{nr}-shot-prompts.pkl')
    preds = [pred.replace(prompts[i], "") for i,pred in enumerate(preds)]

    labels = [l for l in labels if l!=""]
    predictions, num = map_sportstables(preds,prompts)
    
    types = list(set(labels))
    types = types + ["-"] if "-" in predictions else types
    evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)
    
    print(f"{decimal(evaluation['Precision'])}\t{decimal(evaluation['Recall'])}\t{decimal(evaluation['Macro-F1'])}\t{decimal(evaluation['Micro-F1'])}\t{num}")